In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Get table from url, convert to dataframe
def get_data(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  tbl = soup.find("table")
  data_frame = pd.read_html(str(tbl))[0]
  return data_frame

In [3]:
# Get standard grades
def get_standard_grades(year):
  data = get_data(f"https://diemthi.tuyensinh247.com/diem-chuan/dai-hoc-bach-khoa-ha-noi-BKA.html?y={year}")
  idl = [i for i in range(len(list(data[0]))) if list(data[0])[i].isnumeric()]
  data = data.iloc[idl]
  data[5] = data[5].fillna("")
  data[5] = data[5].replace(['Toán'],'Math')
  data[5] = data[5].replace(['Tiếng Anh'],'English')
  data = data[np.invert(data[5].str.contains("kttd", na=False, case=False))]
  data.set_axis(['ID', 'Major ID', 'Major', 'Subjects Combination', 'Standard Grade', 'Main Subject'], axis=1, inplace=True)
  data = data[['Major ID', 'Major', 'Subjects Combination', 'Standard Grade', 'Main Subject']]
  return data

In [4]:
# Year 2020 have main subject -> get main subject from standard_grades[2020]
standard_grades = {}
standard_grades[2020] = get_standard_grades(2020)
main_subject = {}
for x, y in standard_grades[2020].iterrows():
  z = list(y.values)
  main_subject[z[0]] = z[-1]
print(main_subject)

{'BF1': 'Math', 'BF2': 'Math', 'BF-E12': 'Math', 'CH1': 'Math', 'CH2': 'Math', 'CH3': 'Math', 'CH-E11': 'Math', 'ED2': '', 'ΕΕ1': 'Math', 'EE2': 'Math', 'EE-E8': 'Math', 'EE-EP': 'Math', 'EM1': '', 'EM2': '', 'EM3': '', 'EM4': '', 'EM5': '', 'EM-E13': 'Math', 'EM-E14': '', 'EM-VUW': '', 'ET1': 'Math', 'ET-E4': 'Math', 'ET-E5': 'Math', 'ET-E9': 'Math', 'ET-LUH': '', 'EV1': 'Math', 'FL1': 'English', 'FL2': 'English', 'HE1': 'Math', 'IT1': 'Math', 'IT2': 'Math', 'IT-E10': 'Math', 'IT-E6': 'Math', 'IT-E7': 'Math', 'IT-EP': 'Math', 'IT-LTU': '', 'IT-VUW': '', 'ME1': 'Math', 'ME2': 'Math', 'ME-E1': 'Math', 'ME-GU': '', 'ME-LUH': '', 'ME-NUT': '', 'MI1': 'Math', 'MI2': 'Math', 'MS1': 'Math', 'MS-E3': 'Math', 'PH1': 'Math', 'PH2': 'Math', 'TE1': 'Math', 'TE2': 'Math', 'TE3': 'Math', 'TE-E2': 'Math', 'TE-EP': 'Math', 'TROY-BA': '', 'TROY-IT': '', 'TX1': 'Math'}


In [5]:
# Add main subject to standard_grades[2018:2022]
for year in range(2018, 2022):
  standard_grades[year] = get_standard_grades(year)
  new_col = []
  for x, y in standard_grades[year].iterrows():
    z = list(y.values)
    if z[0] in main_subject:    
      new_col.append(main_subject[z[0]])
    else:
      new_col.append('')
  standard_grades[year]['Main Subject'] = new_col
  standard_grades[year]['Subjects Combination'] = standard_grades[year]['Subjects Combination'].str.replace(';', ',')

In [6]:
# Manually fix noises
# Source: '''https://ts.hust.edu.vn/training-cate/nganh-dao-tao-dai-hoc'''
# ED2 (2021): 26.4 -> 24.8
data = standard_grades[2021].copy().set_index('Major ID')
data['Standard Grade']['ED2'] = 24.8
standard_grades[2021]['Standard Grade'] = list(data['Standard Grade'])
display(standard_grades[2021])

,Major ID,Major,Subjects Combination,Standard Grade,Main Subject
1,BF1,Kỹ thuật Sinh học,"A00, B00",25.34,Math
2,BF2,Kỹ thuật Thực phẩm,"A00, B00",25.94,Math
3,CH1,Kỹ thuật Hóa học,"A00, B00, D07",25.2,Math
4,CH2,Hóa học,"A00, B00, D07",24.96,Math
5,CH3,Kỹ thuật In,"A00, B00, D07",24.45,Math
6,ED2,Công nghệ Giáo dục,"A00, A01, D01",24.8,
7,EE1,Kỹ thuật Điện,"A00, A01",26.5,
8,EE2,Kỹ thuật Điều khiển và Tự động hóa,"A00, A01",27.46,Math
9,EM1,Kinh tế Công nghiệp,"A00, A01, D01",25.65,
10,EM2,Quản lý Công nghiệp,"A00, A01, D01",25.75,


In [7]:
# Remove some non float standard grades
def isfloat(x):
  try:
    x = float(x)
    return True
  except:
    return False
for year in range(2018,2022):
  standard_grades[year] = standard_grades[year][standard_grades[year]["Standard Grade"].apply(lambda x: isfloat(x))]
display(standard_grades[2021])

,Major ID,Major,Subjects Combination,Standard Grade,Main Subject
1,BF1,Kỹ thuật Sinh học,"A00, B00",25.34,Math
2,BF2,Kỹ thuật Thực phẩm,"A00, B00",25.94,Math
3,CH1,Kỹ thuật Hóa học,"A00, B00, D07",25.2,Math
4,CH2,Hóa học,"A00, B00, D07",24.96,Math
5,CH3,Kỹ thuật In,"A00, B00, D07",24.45,Math
6,ED2,Công nghệ Giáo dục,"A00, A01, D01",24.8,
7,EE1,Kỹ thuật Điện,"A00, A01",26.5,
8,EE2,Kỹ thuật Điều khiển và Tự động hóa,"A00, A01",27.46,Math
9,EM1,Kinh tế Công nghiệp,"A00, A01, D01",25.65,
10,EM2,Quản lý Công nghiệp,"A00, A01, D01",25.75,


In [8]:
# Export data
for year in range(2018, 2022):
  with open(f'standard_grades_{year}.csv', 'w') as f:
    standard_grades[year].to_csv(f, index=False)